In [3]:
from constant import *
import pandas as pd

In [5]:
file_names = {
    "alter": "XW_ENTINFO_ALTER",
    "basic": "XW_ENTINFO_BASIC",
    "person": "XW_ENTINFO_PERSON",
    "yrpinfo": "XW_ENTINFO_YRPINFO",
    "punished": "XW_ENTINFO_PUNISHED",
    "punishbreak": "XW_ENTINFO_PUNISHBREAK",
    "finalcase": "XW_ENTINFO_FINALCASE",
    "shareholder": "XW_ENTINFO_SHAREHOLDER",
    "taxdeclare": "XW_ENTINFO_TAXDECLARE",
    "fncl_tr_dtal": "XW_ENTINFO_FNCL_TR_DTAL",
    "target": "XW_ENTINFO_TARGET"
}

# 定义一个函数来读取每个文件并检查缺失值和统计量
def load_and_describe_data(dir_path, file_names, suffix):
    data_summary = {}
    
    for key, file_base_name in file_names.items():
        file_name = f"{file_base_name}_{suffix}.csv"
        file_path = os.path.join(dir_path, file_name)
        try:
            # 读取数据
            df = pd.read_csv(file_path)
            # 获取缺失值信息
            missing_info = df.isnull().sum() / len(df) * 100  # 计算缺失百分比
            # 获取基本统计量
            stats = df.describe(include='all')
            
            data_summary[key] = {
                "missing_info": missing_info,
                "statistics": stats
            }
        except Exception as e:
            data_summary[key] = f"Error loading {file_name}: {str(e)}"
    
    return data_summary

# 读取 train 和 test 目录的文件
train_summary = load_and_describe_data(dir_train, file_names, "T")
test_summary = load_and_describe_data(dir_test, file_names, "A")

In [6]:
train_summary

{'alter': {'missing_info': DATA_DAT    0.000000
  CUST_NO     0.000000
  ALTDATE     0.000000
  ALTITEM     4.292961
  dtype: float64,
  'statistics':           DATA_DAT                           CUST_NO       ALTDATE ALTITEM
  count     501705.0                            501705  5.017050e+05  480167
  unique         NaN                             45620           NaN     451
  top            NaN  301499f2d85f25c52129ab0aff9325ec           NaN  经营范围变更
  freq           NaN                               403           NaN   34929
  mean    20020727.0                               NaN  1.996023e+07     NaN
  std            0.0                               NaN  4.758602e+04     NaN
  min     20020727.0                               NaN  1.959083e+07     NaN
  25%     20020727.0                               NaN  1.994022e+07     NaN
  50%     20020727.0                               NaN  1.997080e+07     NaN
  75%     20020727.0                               NaN  2.000011e+07     NaN
  ma

In [7]:
test_summary

{'alter': {'missing_info': DATA_DAT    0.000000
  CUST_NO     0.000000
  ALTDATE     0.000000
  ALTITEM     4.666097
  dtype: float64,
  'statistics':           DATA_DAT                           CUST_NO       ALTDATE ALTITEM
  count      58550.0                             58550  5.855000e+04   55818
  unique         NaN                              5182           NaN     300
  top            NaN  34f4884f034141af87b8821c1bcc5d47           NaN  经营范围变更
  freq           NaN                               285           NaN    4088
  mean    20020827.0                               NaN  1.995922e+07     NaN
  std            0.0                               NaN  4.964756e+04     NaN
  min     20020827.0                               NaN  1.964072e+07     NaN
  25%     20020827.0                               NaN  1.994012e+07     NaN
  50%     20020827.0                               NaN  1.997073e+07     NaN
  75%     20020827.0                               NaN  2.000011e+07     NaN
  ma

缺失值分析：
大多数表中的部分字段存在一定比例的缺失值，例如 XW_ENTINFO_ALTER 表中的 ALTITEM 字段、XW_ENTINFO_SHAREHOLDER 表中的 CONFORM 字段等。不同表的缺失值比例各不相同，最大缺失比例达到了 40% 左右。
金融交易明细表（XW_ENTINFO_FNCL_TR_DTAL）中的某些字段如 ABS_INFO, CPT_CUST_NO 和 SAME_NAM_IND 也存在较高的缺失比例。
数据分布和统计量：
各个表中的数值型字段，如 EXECMONEY, UNPERFMONEY, REGCAP 等，显示了较大的方差，说明这些数值分布存在一定的偏态。
字符串字段（如 COURTNAME, INDS_CD 等）的类别数量较多，这可能需要在后续处理中进行适当的编码或特征提取。
数据一致性与分布差异：
训练和测试集中的数据日期和一些数值型字段（如 REGCAP, SUBCONAM）的统计量差异不大，但某些字段的缺失值比例在两个数据集之间有显著差异，例如 XW_ENTINFO_FINALCASE 表中的 REGDATE 和 UNPERFMONEY 字段。

下一步：
缺失值处理：对于缺失值比例较高的字段，首先需要考虑缺失的原因，并决定是否删除该字段或进行填补。可以考虑：
对数值型字段，使用中位数或平均数填充，或者根据业务逻辑填充。
对类别型字段，考虑使用众数填充，或者为缺失类别单独设置一类。
数据合并与特征工程：根据各个表的结构，通过 CUST_NO 进行合并，构建一个统一的表格。然后可以根据业务逻辑进行特征工程，例如：
计算日期差值，如 成立日期 和 变更日期 之间的差值，作为新的特征。
针对类别型字段，使用目标编码、频率编码等方式进行处理。
数据规范化与编码：对数值特征进行归一化或标准化，防止特征之间的量级差异影响模型训练。对类别特征使用合适的编码方式（如 Label Encoding 或 One-Hot Encoding）。

In [8]:
# 读取目标表
train_target_path = f"{dir_train}/XW_ENTINFO_TARGET_T.csv"
test_target_path = f"{dir_test}/XW_ENTINFO_TARGET_A.csv"

# 读取训练集和测试集的目标表
train_target = pd.read_csv(train_target_path)
test_target = pd.read_csv(test_target_path)

# 计算训练集的正负样本比例
train_pos_samples = train_target[train_target['FLAG'] == 1].shape[0]
train_neg_samples = train_target[train_target['FLAG'] == 0].shape[0]
train_total_samples = train_target.shape[0]

# 计算比例
train_pos_ratio = train_pos_samples / train_total_samples
train_neg_ratio = train_neg_samples / train_total_samples

# 打印结果
print(f"训练集中的正样本比例: {train_pos_ratio:.4f}")
print(f"训练集中的负样本比例: {train_neg_ratio:.4f}")

训练集中的正样本比例: 0.0438
训练集中的负样本比例: 0.9562


In [10]:
# 检查各个表的 DATA_DAT 字段的唯一值
def check_data_dat_consistency(dir_path, file_names, suffix):
    data_dat_values = {}
    
    for key, file_name in file_names.items():
        file_path = os.path.join(dir_path,f'{file_name}_{suffix}.csv')
        try:
            # 读取数据
            df = pd.read_csv(file_path, usecols=["DATA_DAT"])  # 只读取 DATA_DAT 字段
            unique_values = df["DATA_DAT"].unique()
            data_dat_values[key] = unique_values
        except Exception as e:
            data_dat_values[key] = f"Error reading {file_name}: {str(e)}"
    
    return data_dat_values

# 执行函数并检查结果
data_dat_values = check_data_dat_consistency(dir_train, file_names, 'T')
for table, values in data_dat_values.items():
    print(f"{table}: {values}")
    
data_dat_values = check_data_dat_consistency(dir_test, file_names,'A')
for table, values in data_dat_values.items():
    print(f"{table}: {values}")

alter: [20020727]
basic: [20020727]
person: [20020727]
yrpinfo: [20020727]
punished: [20020727]
punishbreak: [20020727]
finalcase: [20020727]
shareholder: [20020727]
taxdeclare: [20020727]
fncl_tr_dtal: Error reading XW_ENTINFO_FNCL_TR_DTAL: Usecols do not match columns, columns expected but not found: ['DATA_DAT']
target: [20030728]
alter: [20020827]
basic: [20020827]
person: [20020827]
yrpinfo: [20020827]
punished: [20020827]
punishbreak: [20020827]
finalcase: [20020827]
shareholder: [20020827]
taxdeclare: [20020827]
fncl_tr_dtal: Error reading XW_ENTINFO_FNCL_TR_DTAL: Usecols do not match columns, columns expected but not found: ['DATA_DAT']
target: [20030828]
